FOREX data source for manual downloading: https://www.dukascopy.com/plugins/fxMarketWatch/?historical_data 

# 3. Download and Process FX rates

In [6]:
import pandas as pd
import numpy as np


def load_prices(pairname, period, start_date, end_date):
    data = pd.DataFrame()
    if pairname == 'EURUSD':
        data = pd.read_csv('Data/Testing_data/Prices/' +
                              'EURUSD_Candlestick_1_' + period + '_BID_' + start_date + '-' + end_date + '.csv')
        data = data[['Gmt time', 'Close']]
        data.rename(columns = {'Close' : 'EURUSD'}, inplace = True)
    if pairname == 'GBPUSD':
        data = pd.read_csv('Data/Testing_data/Prices/' +
                              'GBPUSD_Candlestick_1_' + period + '_BID_' + start_date + '-' + end_date + '.csv')
        data = data[['Gmt time', 'Close']]
        data.rename(columns = {'Close' : 'GBPUSD'}, inplace = True)
    if pairname == 'USDCAD':
        data = pd.read_csv('Data/Testing_data/Prices/' +
                              'USDCAD_Candlestick_1_' + period + '_BID_' + start_date + '-' + end_date + '.csv')
        data = data[['Gmt time', 'Close']]
        data['Close'] = 1 / data['Close']
        data.rename(columns = {'Close' : 'CADUSD'}, inplace = True)
    if pairname == 'USDJPY':
        data = pd.read_csv('Data/Testing_data/Prices/' +
                              'USDJPY_Candlestick_1_' + period + '_BID_' + start_date + '-' + end_date + '.csv')
        data = data[['Gmt time', 'Close']]
        data['Close'] = 1 / data['Close']
        data.rename(columns = {'Close' : 'JPYUSD'}, inplace = True)
    return data

In [7]:
start_date = '06.01.2019'
end_date = '30.05.2020'

eurusd_m = load_prices('EURUSD', 'M', start_date, end_date)
eurusd_h = load_prices('EURUSD', 'Hour', start_date, end_date)
eurusd_d = load_prices('EURUSD', 'D', start_date, end_date)

cadusd_m = load_prices('USDCAD', 'M', start_date, end_date)
cadusd_h = load_prices('USDCAD', 'Hour', start_date, end_date)
cadusd_d = load_prices('USDCAD', 'D', start_date, end_date)

gbpusd_m = load_prices('GBPUSD', 'M', start_date, end_date)
gbpusd_h = load_prices('GBPUSD', 'Hour', start_date, end_date)
gbpusd_d = load_prices('GBPUSD', 'D', start_date, end_date)

jpyusd_m = load_prices('USDJPY', 'M', start_date, end_date)
jpyusd_h = load_prices('USDJPY', 'Hour', start_date, end_date)
jpyusd_d = load_prices('USDJPY', 'D', start_date, end_date)

In [8]:
data_m1 = pd.merge(eurusd_m, cadusd_m, on=['Gmt time'])
data_m2 = pd.merge(gbpusd_m, jpyusd_m, on=['Gmt time'])
data_m = pd.merge(data_m1, data_m2, on=['Gmt time'])

data_h1 = pd.merge(eurusd_h, cadusd_h, on=['Gmt time'])
data_h2 = pd.merge(gbpusd_h, jpyusd_h, on=['Gmt time'])
data_h = pd.merge(data_h1, data_h2, on=['Gmt time'])

data_d1 = pd.merge(eurusd_d, cadusd_d, on=['Gmt time'])
data_d2 = pd.merge(gbpusd_d, jpyusd_d, on=['Gmt time'])
data_d = pd.merge(data_d1, data_d2, on=['Gmt time'])

In [ ]:
from datetime import datetime
from tqdm import tnrange

for i in tnrange(data_m.shape[0]):
    data_m['Gmt time'][i] = datetime.strptime(data_m['Gmt time'][i][:19], '%d.%m.%Y %H:%M:%S') # , '%Y-%m-%d %H:%m:%s'
data_m.to_csv('Testing_data/Prices/data_m.csv', header=True, index=False)
data_m.head(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [21]:
for i in tnrange(data_h.shape[0]):
    data_h['Gmt time'][i] = datetime.strptime(data_h['Gmt time'][i][:19], '%d.%m.%Y %H:%M:%S') # , '%Y-%m-%d %H:%m:%s'
data_h.to_csv('Testing_data/Prices/data_h.csv', header=True, index=False)
data_h.head(5)

In [22]:
for i in tnrange(data_d.shape[0]):
    data_d['Gmt time'][i] = datetime.strptime(data_d['Gmt time'][i][:19], '%d.%m.%Y %H:%M:%S') # , '%Y-%m-%d %H:%m:%s'
data_d.to_csv('Testing_data/Prices/data_d.csv', header=True, index=False)
data_d.head(5)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Gmt time,EURUSD,CADUSD,GBPUSD,JPYUSD
0,2005-09-08 21:00:00,1.24064,0.849495,1.83929,0.009120
1,2005-09-11 21:00:00,1.22803,0.842737,1.81932,0.009064
2,2005-09-12 21:00:00,1.22625,0.846819,1.82204,0.009040
3,2005-09-13 21:00:00,1.22813,0.847501,1.82336,0.009060
4,2005-09-14 21:00:00,1.22229,0.844480,1.80561,0.009041


## Calculate exchange rate relatives

In [23]:
data_m2 = data_m.drop(['Gmt time'], axis = 1)
data_m3 = (data_m2.shift(1) / data_m2).dropna() # returns
data_m4 = data_m[['Gmt time']] # date
data_m_ret = pd.merge(data_m3, data_m4, left_index=True, right_index=True)
print(data_m.shape)
print(data_m_ret.shape)
data_m_ret.to_csv('Testing_data/Prices/data_m_ret.csv', header=True, index=False)

In [24]:
data_h2 = data_h.drop(['Gmt time'], axis = 1)
data_h3 = (data_h2.shift(1) / data_h2).dropna() # returns
data_h4 = data_h[['Gmt time']] # date
data_h_ret = pd.merge(data_h3, data_h4, left_index=True, right_index=True)
print(data_h.shape)
print(data_h_ret.shape)
data_h_ret.to_csv('Testing_data/Prices/data_h_ret.csv', header=True, index=False)

In [25]:
data_d2 = data_d.drop(['Gmt time'], axis = 1)
data_d3 = (data_d2.shift(1) / data_d2).dropna() # returns
data_d4 = data_d[['Gmt time']] # date
data_d_ret = pd.merge(data_d3, data_d4, left_index=True, right_index=True)
print(data_d.shape)
print(data_d_ret.shape)
data_d_ret.to_csv('Testing_data/Prices/data_d_ret.csv', header=True, index=False)

(1055, 5)
(1054, 5)
